<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 533.1/533.1 kB 7.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=c68801c48521d312d05f0cab8d7254cdb107f2f8d95ec49bd728a4977f1b796f
  Stored in directory: /root/.cache/pip/wheels/5c/a1/5f/c6b85a7d9452057be4ce68a8e45d77ba34234a6d46581777c6
Successfully built ta
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.4/117.4 kB 3.4 MB/s eta 0:00:00
  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-13 18:54:35
-------------------
qualified stocks: 87
with latest results: 26
still star stocks: 16
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  3.00 K
Current:  1.45 C
-------------------
Today PnL: -49.04 K (-0.34%)
Current PnL: -20.15 L (-13.21%)
CY Booked + Current PnL: -6.63 L (-4.35%)
-------------------
Total profit:  1.90 L
Total loss:  -22.06 L
-------------------
Total Booked + Current PnL: 20.35 L (16.2%)
Total Booked PnL: 40.50 L (32.24%)
Curr Year Booked PnL: 13.52 L (9.31%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.36 C
Est FTT PnL: 91.05 L (62.68%)
Deployed:  1.26 C
Current:  1.45 C
CAGR/XIRR %: 8.45%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TTKPRESTIG,770.00,100.64,50.0,M-SC,2.67,86408.0,-14369.0,14465.0,-0.66,-14.26,16.74,0.09,253.0,-0.99,0.59,13.52,OX40N,NTT,DURABLES
39,INDIGOPNTS,1408.00,143.86,83.0,M-SC,3.12,160010.0,-14549.0,14577.0,4.70,-8.33,9.11,0.02,234.0,-1.00,1.10,39.18,OX40N,NTT,PAINTS
2,ABBOTINDIA,35195.00,-15.14,47.0,X-MC,3.35,88710.0,-1968.0,16873.0,0.00,-2.17,19.02,16.44,63.0,-0.12,0.61,17.57,X40,ATH,PHARMA
35,ICICIGI,2252.93,-15.07,59.0,X-MC,7.58,189143.0,14823.0,20371.0,0.26,8.50,10.77,20.19,68.0,0.73,1.30,24.90,X40,ATH,INSURANCE
43,ITC,452.00,-40.74,44.0,X-LC,0.87,198387.0,-1751.0,22636.0,-0.32,-0.87,11.41,10.44,5.0,-0.08,1.37,4.64,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,143.86,83.0,M-SC,3.12,160010.0,-14549.0,14577.0,4.70,-8.33,9.11,0.02,234.0,-1.00,1.10,39.18,OX40N,NTT,PAINTS
6,ASIANPAINT,3465.66,-0.35,86.0,X-LC,7.98,253387.0,1619.0,51590.0,3.96,0.64,20.36,21.13,31.0,0.03,1.74,36.30,X40,ATH,PAINTS
13,BERGEPAINT,680.00,-4.29,70.0,X-MC,1.63,234050.0,6711.0,41357.0,3.18,2.95,17.67,21.14,74.0,0.16,1.61,31.92,XY24,NTT,PAINTS
56,RAJESHEXPO,518.00,1697.98,56.0,L-SC,3.48,49046.0,-88131.0,88224.0,2.55,-64.25,179.88,0.07,268.0,-1.00,0.34,21.98,OX40N,NTT,JEWELLERY
47,KANSAINER,340.00,-65.79,52.0,H-SC,2.40,224406.0,-45261.0,81594.0,1.55,-16.78,36.36,13.47,158.0,-0.55,1.55,14.10,XY24,NTT,PAINTS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,REPCOHOME,880.00,-55.29,53.0,H-SC,9.49,258448.0,-28057.0,285404.0,-4.86,-9.79,110.43,89.82,160.0,-0.10,1.78,36.74,XY24,NTT,FINANCE
48,KPIGREEN,731.05,-12.12,38.0,H-SC,10.17,118957.0,-6340.0,65272.0,-2.85,-5.06,54.87,47.03,154.0,-0.10,0.82,48.94,MH,ATH,POWER
53,PAGEIND,51769.93,-28.83,36.0,X-MC,7.65,158340.0,-5400.0,48737.0,-2.79,-3.30,30.78,26.47,55.0,-0.11,1.09,0.32,X40,ATH,APPARELS
7,ASIANTILES,137.00,7505.56,62.0,L-SC,15.79,83435.0,-10375.0,86989.0,-2.30,-11.06,104.26,81.67,271.0,-0.12,0.57,61.58,XR,NTT,CERAMICS
55,QUESS,986.00,-52.62,30.0,X-SC,37.28,47050.0,-17956.0,168886.0,-2.08,-27.62,358.95,232.18,83.0,-0.11,0.32,0.00,XY24,NTT,MISC


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-18.52,47.0,M-SC,10.41,147599.0,-45.0,159377.0,0.11,-0.03,107.98,107.91,216.0,-0.00,1.02,2.97,OX40N,ATH,CABLES
86,ZYDUSLIFE,1286.17,-14.53,37.0,H-MC,5.89,208657.0,-917.0,75596.0,-0.76,-0.44,36.23,35.63,119.0,-0.01,1.44,16.16,AR,ATH,PHARMA
85,WIPRO,420.00,-13.79,55.0,M-LC,5.45,152105.0,1160.0,108299.0,0.05,0.77,71.20,72.51,99.0,0.01,1.05,6.84,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-18.52,47.0,M-SC,10.41,147599.0,-45.0,159377.0,0.11,-0.03,107.98,107.91,216.0,-0.00,1.02,2.97,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,143.86,83.0,M-SC,3.12,160010.0,-14549.0,14577.0,4.70,-8.33,9.11,0.02,234.0,-1.00,1.10,39.18,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,100.64,50.0,M-SC,2.67,86408.0,-14369.0,14465.0,-0.66,-14.26,16.74,0.09,253.0,-0.99,0.59,13.52,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-65.79,52.0,H-SC,2.40,224406.0,-45261.0,81594.0,1.55,-16.78,36.36,13.47,158.0,-0.55,1.55,14.10,XY24,NTT,PAINTS
67,SIS,528.00,2022.28,47.0,H-SC,2.14,85654.0,-25378.0,48986.0,-0.30,-22.86,57.19,21.26,166.0,-0.52,0.59,15.73,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,SAMMAANCAP,326.00,18.70,56.0,M-SC,36.10,160497.0,11277.0,132908.0,0.77,7.56,82.81,96.62,245.0,0.08,1.11,73.69,XY25,NTT,FINANCE
85,WIPRO,420.00,-13.79,55.0,M-LC,5.45,152105.0,1160.0,108299.0,0.05,0.77,71.20,72.51,99.0,0.01,1.05,6.84,XR,NTT,IT
38,INDIAMART,4810.62,-47.39,56.0,H-SC,4.76,131286.0,7950.0,123671.0,-1.09,6.45,94.20,106.72,139.0,0.06,0.90,31.33,AR,ATH,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIAMART,4810.62,-47.39,56.0,H-SC,4.76,131286.0,7950.0,123671.0,-1.09,6.45,94.20,106.72,139.0,0.06,0.90,31.33,AR,ATH,MISC
85,WIPRO,420.00,-13.79,55.0,M-LC,5.45,152105.0,1160.0,108299.0,0.05,0.77,71.20,72.51,99.0,0.01,1.05,6.84,XR,NTT,IT
25,FINCABLES,1641.55,-18.52,47.0,M-SC,10.41,147599.0,-45.0,159377.0,0.11,-0.03,107.98,107.91,216.0,-0.00,1.02,2.97,OX40N,ATH,CABLES
86,ZYDUSLIFE,1286.17,-14.53,37.0,H-MC,5.89,208657.0,-917.0,75596.0,-0.76,-0.44,36.23,35.63,119.0,-0.01,1.44,16.16,AR,ATH,PHARMA
84,WHIRLPOOL,2270.00,-42.35,36.0,M-SC,9.18,112912.0,-1886.0,95930.0,0.03,-1.64,84.96,81.92,236.0,-0.02,0.78,33.67,XR,NTT,DURABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,GILLETTE,11206.78,-17.85,28.0,X-SC,5.77,253905.0,9.0,82291.0,-0.54,0.00,32.41,32.42,85.0,0.00,1.75,15.10,X40,ATH,FMCG
27,GLAXO,3466.20,-19.65,29.0,X-MC,5.35,199560.0,276.0,77729.0,-0.49,0.14,38.95,39.15,60.0,0.00,1.37,29.18,X40,ATH,PHARMA
12,BATAINDIA,2096.00,13.94,30.0,X-SC,8.94,86851.0,-41819.0,87120.0,-1.05,-32.50,100.31,35.20,92.0,-0.48,0.60,0.00,X40,NTT,FOOTWEAR
55,QUESS,986.00,-52.62,30.0,X-SC,37.28,47050.0,-17956.0,168886.0,-2.08,-27.62,358.95,232.18,83.0,-0.11,0.32,0.00,XY24,NTT,MISC
54,PGHH,17907.65,-31.97,30.0,X-MC,1.55,193545.0,-7275.0,75076.0,0.25,-3.62,38.79,33.76,57.0,-0.10,1.33,1.39,X40,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TMPV,1065.00,-15.75,44.0,X-LC,0.07,260657.0,-15753.0,436913.0,-1.04,-5.70,167.62,152.37,3.0,-0.04,1.79,1.82,XY24,NTT,AUTO
36,ICICIPRULI,790.00,-20.11,67.0,X-MC,0.77,184619.0,7881.0,46856.0,-0.45,4.46,25.38,30.97,75.0,0.17,1.27,17.58,X40,ATH,INSURANCE
43,ITC,452.00,-40.74,44.0,X-LC,0.87,198387.0,-1751.0,22636.0,-0.32,-0.87,11.41,10.44,5.0,-0.08,1.37,4.64,X40,NTT,FMCG
34,HONAUT,58357.33,-25.10,36.0,X-SC,0.97,106545.0,-21393.0,68530.0,0.06,-16.72,64.32,36.84,90.0,-0.31,0.73,9.08,X40N,ATH,ELECTRICAL
16,CAMS,4762.00,-1.47,57.0,X-SC,1.44,260304.0,8448.0,53987.0,-0.50,3.35,20.74,24.79,86.0,0.16,1.79,27.27,X40N,NTT,MISC


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BATAINDIA,2096.00,13.94,30.0,X-SC,8.94,86851.0,-41819.0,87120.0,-1.05,-32.50,100.31,35.20,92.0,-0.48,0.60,0.00,X40,NTT,FOOTWEAR
55,QUESS,986.00,-52.62,30.0,X-SC,37.28,47050.0,-17956.0,168886.0,-2.08,-27.62,358.95,232.18,83.0,-0.11,0.32,0.00,XY24,NTT,MISC
53,PAGEIND,51769.93,-28.83,36.0,X-MC,7.65,158340.0,-5400.0,48737.0,-2.79,-3.30,30.78,26.47,55.0,-0.11,1.09,0.32,X40,ATH,APPARELS
54,PGHH,17907.65,-31.97,30.0,X-MC,1.55,193545.0,-7275.0,75076.0,0.25,-3.62,38.79,33.76,57.0,-0.10,1.33,1.39,X40,ATH,FMCG
10,BAJAJHFL,181.50,-18.37,37.0,X-MC,6.43,176530.0,-23976.0,121135.0,-0.36,-11.96,68.62,48.45,64.0,-0.20,1.22,1.80,X40N,ATH,FINANCE


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,986.00,-52.62,30.0,X-SC,37.28,47050.0,-17956.0,168886.0,-2.08,-27.62,358.95,232.18,83.0,-0.11,0.32,0.00,XY24,NTT,MISC
59,RELAXO,1176.00,-40.76,42.0,X-SC,3.58,80755.0,-64405.0,140328.0,-1.16,-44.37,173.77,52.31,91.0,-0.46,0.56,8.10,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,13.94,30.0,X-SC,8.94,86851.0,-41819.0,87120.0,-1.05,-32.50,100.31,35.20,92.0,-0.48,0.60,0.00,X40,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-15.14,47.0,X-MC,3.35,88710.0,-1968.0,16873.0,0.00,-2.17,19.02,16.44,63.0,-0.12,0.61,17.57,X40,ATH,PHARMA
34,HONAUT,58357.33,-25.10,36.0,X-SC,0.97,106545.0,-21393.0,68530.0,0.06,-16.72,64.32,36.84,90.0,-0.31,0.73,9.08,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4389.97,-26.54,59.0,X-LC,8.78,291936.0,-54020.0,120716.0,-0.83,-15.61,41.35,19.28,1.0,-0.45,2.01,7.92,X40,ATH,IT
41,INFY,2275.00,-18.16,64.0,X-LC,4.33,328403.0,15515.0,156156.0,-0.64,4.96,47.55,54.87,2.0,0.10,2.26,13.65,X40,BTT,IT
76,TMPV,1065.00,-15.75,44.0,X-LC,0.07,260657.0,-15753.0,436913.0,-1.04,-5.70,167.62,152.37,3.0,-0.04,1.79,1.82,XY24,NTT,AUTO
82,VBL,671.64,-19.36,43.0,X-LC,5.88,289014.0,-26828.0,139478.0,-1.37,-8.49,48.26,35.67,4.0,-0.19,1.99,4.18,X40N,ATH,FMCG
43,ITC,452.00,-40.74,44.0,X-LC,0.87,198387.0,-1751.0,22636.0,-0.32,-0.87,11.41,10.44,5.0,-0.08,1.37,4.64,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,-708.10,78.0,L-MC,6.71,49217.0,-27990.0,53386.0,-0.11,-36.25,108.47,32.89,259.0,-0.52,0.34,35.54,XR,NTT,BANKS
19,COFFEEDAY,80.00,-47.69,45.0,L-SC,22.25,77626.0,-35923.0,75973.0,-0.05,-31.64,97.87,35.27,270.0,-0.47,0.53,89.99,XR,NTT,HOTELS
7,ASIANTILES,137.00,7505.56,62.0,L-SC,15.79,83435.0,-10375.0,86989.0,-2.30,-11.06,104.26,81.67,271.0,-0.12,0.57,61.58,XR,NTT,CERAMICS
51,MASFIN,398.61,-19.15,49.0,H-SC,7.51,91875.0,-6105.0,27710.0,-0.57,-6.23,30.16,22.05,164.0,-0.22,0.63,33.55,XR,ATH,FINANCE
38,INDIAMART,4810.62,-47.39,56.0,H-SC,4.76,131286.0,7950.0,123671.0,-1.09,6.45,94.20,106.72,139.0,0.06,0.90,31.33,AR,ATH,MISC


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,143.86,83.0,M-SC,3.12,160010.0,-14549.0,14577.0,4.70,-8.33,9.11,0.02,234.0,-1.00,1.10,39.18,OX40N,NTT,PAINTS
6,ASIANPAINT,3465.66,-0.35,86.0,X-LC,7.98,253387.0,1619.0,51590.0,3.96,0.64,20.36,21.13,31.0,0.03,1.74,36.30,X40,ATH,PAINTS
80,VAIBHAVGBL,521.00,72.45,61.0,H-SC,7.54,149629.0,-33146.0,144212.0,0.15,-18.14,96.38,60.77,159.0,-0.23,1.03,34.75,XR,NTT,JEWELLERY
7,ASIANTILES,137.00,7505.56,62.0,L-SC,15.79,83435.0,-10375.0,86989.0,-2.30,-11.06,104.26,81.67,271.0,-0.12,0.57,61.58,XR,NTT,CERAMICS
5,ANGELONE,3033.00,58.43,72.0,X-SC,7.07,222507.0,31501.0,26189.0,1.27,16.49,11.77,30.21,97.0,1.20,1.53,40.82,X40N,NTT,FINANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.54
1,25,44.40
2,50,75.63


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

,CurrAlloc%
Conviction,
SC,47.58
MC,27.67
LC,24.74


In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

,CurrAlloc%
Criteria,
XY24,27.42
X40,20.55
X40N,12.21
XY25,10.04
XR,9.59
AR,8.98
OX40N,7.54
X200,1.81
SR,1.03


In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

,CurrAlloc%
Conviction,
H-SC,24.18
X-MC,21.32
X-LC,19.84
M-SC,13.41
X-SC,8.55
H-MC,4.65
H-LC,2.87
L-SC,1.44
M-MC,1.36


In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.58,-4.51,38.31
IT,12.59,-14.96,76.49
FINANCE,11.89,-9.55,59.21
MISC,7.55,-18.18,77.78
PAINTS,6.00,-5.90,21.69
ELECTRICAL,5.53,-12.46,52.32
INSURANCE,4.45,2.17,32.30
AUTO,3.49,-13.41,122.13
PHARMA,3.42,-0.53,34.25


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3323183.0,20
AR,1266423.0,10
XR,1263095.0,13
X40,983577.0,14
OX40N,696877.0,10
X40N,695426.0,9
XY25,477846.0,7
SR,271990.0,2
MH,65272.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3491946.0,25
M-SC,1406893.0,16
X-LC,1185613.0,11
X-MC,1144339.0,15
X-SC,833451.0,8
H-MC,402876.0,3
L-SC,251186.0,3
H-LC,141234.0,2
M-LC,108299.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1182764.0      6
           AR         841129.0      5
           XR         795020.0      7
M-SC       XY24       733255.0      6
X-LC       XY24       502538.0      2
X-MC       X40        412273.0      7
X-LC       X40        401893.0      5
X-SC       XY24       375006.0      2
X-MC       XY24       347348.0      3
H-SC       OX40N      335771.0      4
X-SC       X40N       289034.0      4
M-SC       OX40N      272882.0      5
H-SC       SR         271990.0      2
H-MC       AR         220604.0      2
X-LC       X40N       205920.0      2
X-MC       X40N       200472.0      3
           XY25       184246.0      2
H-MC       XY24       182272.0      1
X-SC       X40        169411.0      2
L-SC       XR         162962.0      2
M-SC       XR         143428.0      2
           XY25       132908.0      1
           AR         124420.0      2
M-LC       XR         108299.0      1
L-SC       OX40N       88224.0      1
H-LC       AR          80270.0      1
X-LC       XY25        75262.0      2
H-SC       MH          65272.0      1
H-LC       X200        60964.0      1
L-MC       XR          53386.0      1
M-MC       XY25        47829.0      1
L-LC       XY25        37601.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 50.0 seconds
